In [10]:
import pandas as pd
import numpy as np
import sqlite3


In [6]:
con = sqlite3.connect("../data/my_history-2022-10-14.sqlite")

df = pd.read_sql_query("SELECT * from urls", con)

In [112]:
import re

# Match https://*/
regex_extract_domain = re.compile('^[^/]+://([^/]+/?[^/\?]*)')

# Match file:///* and data:image/*
regex_extract_domain_temp = re.compile('^([^/]+:/+[^/]+/?[^/\?]*|data:[^/]+)')
def extract_domain_with_regex(url: str):
  m = regex_extract_domain.match(url)
  try:
    domain = m.group(1)
  except AttributeError:
    m2 = regex_extract_domain_temp.match(url)

    try:
      domain = m2.group(1)
    except AttributeError:
      domain = "ERROR"

  return domain

In [113]:
def extract_domain_with_string_funcs(url:str):
  start = url.find('//') + 2
  end = url.find('/', start)
  return url[start:end]

In [116]:
def extract_domain(url:str):
  return extract_domain_with_string_funcs(url)

In [127]:
map_domains = {
  'google.ca/maps': 'google.com/maps',
  'google.ca': 'google.com',
}

def process_domain(url:str):
  domain : str = extract_domain(url)
  domain = domain.strip('/')
  domain = domain.removeprefix('www.').removeprefix('www2.')

  if (domain in map_domains):
    return map_domains[domain]
  else:
    return domain
  

df['domain'] = df['url'].apply(process_domain)


In [128]:
# https://stackoverflow.com/a/65784434
relevant_fields =       ['visit_count', 'url', 'title', 'domain']
relevant_fields_alias = ['visit_count', 'sample_url', 'sample_title']

df_grouped_count = df[relevant_fields].groupby('domain').agg(np.sum)
top_50 = df_grouped_count.sort_values('visit_count', ascending=False)[0:50]

top_50.to_csv('../output/top_50_plain.csv')

df_grouped_count__with_first = df[relevant_fields].groupby('domain').agg({k: np.sum if k == 'visit_count' else 'first' for k in relevant_fields})
top_50__with_first = df_grouped_count__with_first[relevant_fields[0:3]].sort_values('visit_count', ascending=False)[0:50]

top_50__with_first.to_csv('../output/top_50.csv', header=relevant_fields_alias)

In [129]:
df[df['domain'] == 'cs.sfu.ca']

,id,url,title,visit_count,typed_count,last_visit_time,hidden,domain
724,71215,https://www.cs.sfu.ca/~wsumner/teaching/373,CMPT 373: Software Development Methods,1,0,13307829283496862,0,cs.sfu.ca
12139,86872,https://www2.cs.sfu.ca/~wsumner/teaching/473/,"CMPT 473: Software Testing, Reliability and Se...",1,0,13306902305707584,0,cs.sfu.ca
12140,86873,http://www.cs.sfu.ca/~wsumner/teaching/473/22,"CMPT 473: Software Testing, Reliability and Se...",1,0,13306902305707584,0,cs.sfu.ca
12141,86874,https://www2.cs.sfu.ca/~wsumner/teaching/473/22,"CMPT 473: Software Testing, Reliability and Se...",1,0,13306902305707584,0,cs.sfu.ca
12142,86875,https://www2.cs.sfu.ca/~wsumner/teaching/473/22/,"CMPT 473: Software Testing, Reliability and Se...",37,4,13310183446993298,0,cs.sfu.ca
12144,86877,https://www2.cs.sfu.ca/~wsumner/teaching/473/2...,Schedule,19,0,13310183598677073,0,cs.sfu.ca
12148,86881,https://www2.cs.sfu.ca/~wsumner/teaching/473/0...,Course Slides,3,0,13309458099066888,0,cs.sfu.ca
12149,86882,https://www2.cs.sfu.ca/~wsumner/teaching/473/0...,Course Slides,3,0,13309458102854560,0,cs.sfu.ca
12155,86888,https://www2.cs.sfu.ca/~wsumner/teaching/373/2...,Schedule,21,0,13310172526836901,0,cs.sfu.ca
12156,86889,https://www2.cs.sfu.ca/~wsumner/teaching/473/2...,Exercises,13,0,13310117851806640,0,cs.sfu.ca
